<a href="https://colab.research.google.com/github/marlonsmacedo/Santander-Bootcamp_IA-Generativa-ETL-Python/blob/main/PipelineETLPython_BootcampDIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

Baseado nas premissas do desafio, segue código com algumas alterações:

### Extract:
Obter lista de Usuários da API Santander e Adicioná-los à um dataframe e salvá-los em um arquivo excel.








In [85]:
!pip install requests openai xlwt


In [74]:
import pandas as pd
import requests
import sqlite3
import json

# API para get de usuários:
API_URL = 'https://sdw-2023-prd.up.railway.app/users'


# obter lista de users da API Santander:
def get_users(id):

    response = requests.get(f'{API_URL}/{id}')
    return response.json() if response.status_code == 200 else None

# Iterar 50 ids e pular requisições vazias de id que não existam e retornem algo diferente de status 200 e adicionar à uma lista:
users_json = []

for i in range(50):
    if get_users(i) == None:
        continue
    else:
        users_json.append(get_users(i))

print(json.dumps(users_json, indent=2))
df = pd.DataFrame(users_json)
df.head()


[
  {
    "id": 1,
    "name": "Devweekerson",
    "account": {
      "id": 1,
      "number": "01.097954-4",
      "agency": "2030",
      "balance": 624.12,
      "limit": 1000.0
    },
    "card": {
      "id": 1,
      "number": "xxxx xxxx xxxx 1111",
      "limit": 2000.0
    },
    "features": [
      {
        "id": 1,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pix.svg",
        "description": "PIX"
      },
      {
        "id": 2,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
        "description": "Pagar"
      },
      {
        "id": 3,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
        "description": "Transferir"
      },
      {
        "id": 4,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
        "description": "Conta Corrente"
      },
      {
        

,id,name,account,card,features,news
0,1,Devweekerson,"{'id': 1, 'number': '01.097954-4', 'agency': '...","{'id': 1, 'number': 'xxxx xxxx xxxx 1111', 'li...","[{'id': 1, 'icon': 'https://digitalinnovationo...","[{'id': 1, 'icon': 'https://digitalinnovationo..."
1,27,Predu,"{'id': 34, 'number': '99999-07', 'agency': '00...","{'id': 27, 'number': '**** **** **** 7777', 'l...",[],"[{'id': 170, 'icon': None, 'description': 'Pre..."
2,28,Pamonha,"{'id': 35, 'number': '100000-08', 'agency': '0...","{'id': 28, 'number': '**** **** **** 8888', 'l...",[],"[{'id': 171, 'icon': None, 'description': 'Oi ..."
3,29,Faustão,"{'id': 36, 'number': '100000-09', 'agency': '0...","{'id': 29, 'number': '**** **** **** 9999', 'l...",[],"[{'id': 172, 'icon': None, 'description': 'Olá..."
4,65,devteste-9997,"{'id': 72, 'number': 'number-9997', 'agency': ...","{'id': 65, 'number': 'number-card-9997', 'limi...","[{'id': 33, 'icon': 'string', 'description': '...","[{'id': 1022, 'icon': 'https://digitalinnovati..."



## Transform

Se reparar o resulado do dataframe acima os dados das colunas estão alimentados com listas e dicionarios provenientes da conversão.

Para Salvar os arquivos em um Excel, primeiro precisamos Normalizar os dados, pois os campos possuem dados aninhados

### Normalizando a coluna "account":

In [75]:
# Vamos normalizar Coluna "account" usando a função json_normalize() do Pandas:
account_df = pd.json_normalize(df['account'])

account_df.head()

,id,number,agency,balance,limit
0,1,01.097954-4,2030,624.12,1000.0
1,34,99999-07,0007,4000.00,10000.0
2,35,100000-08,0007,4000.00,10000.0
3,36,100000-09,0007,4000.00,10000.0
4,72,number-9997,ag-9997,1230.00,1230.0


Com a função `json_normalize()` do Pandas trasformamos os dados aninhados em uma tabela, agora precisamos incluir esses dados no nosso Dataframe, para isso vamos usar a função `concat()`do Pandas. Dentro da função `concat()` vamos excluir a coluna account que está aninhada:

In [76]:
df = pd.concat([df.drop(['account'], axis=1), account_df], axis=1)
df.head()

,id,name,card,features,news,id,number,agency,balance,limit
0,1,Devweekerson,"{'id': 1, 'number': 'xxxx xxxx xxxx 1111', 'li...","[{'id': 1, 'icon': 'https://digitalinnovationo...","[{'id': 1, 'icon': 'https://digitalinnovationo...",1,01.097954-4,2030,624.12,1000.0
1,27,Predu,"{'id': 27, 'number': '**** **** **** 7777', 'l...",[],"[{'id': 170, 'icon': None, 'description': 'Pre...",34,99999-07,0007,4000.00,10000.0
2,28,Pamonha,"{'id': 28, 'number': '**** **** **** 8888', 'l...",[],"[{'id': 171, 'icon': None, 'description': 'Oi ...",35,100000-08,0007,4000.00,10000.0
3,29,Faustão,"{'id': 29, 'number': '**** **** **** 9999', 'l...",[],"[{'id': 172, 'icon': None, 'description': 'Olá...",36,100000-09,0007,4000.00,10000.0
4,65,devteste-9997,"{'id': 65, 'number': 'number-card-9997', 'limi...","[{'id': 33, 'icon': 'string', 'description': '...","[{'id': 1022, 'icon': 'https://digitalinnovati...",72,number-9997,ag-9997,1230.00,1230.0


In [77]:
# Vamos normalizar Coluna "card" usando a função json_normalize() do Pandas:
card_df = pd.json_normalize(df['card'])
# Renomear as colunas dos dados para identificar que são dados do cartão:
card_df.rename(columns={'id':'card_id', 'number':'card_number', 'limit':'card_limit'}, inplace=True)
card_df.head()



,card_id,card_number,card_limit
0,1,xxxx xxxx xxxx 1111,2000.0
1,27,**** **** **** 7777,10000.0
2,28,**** **** **** 8888,10000.0
3,29,**** **** **** 9999,10000.0
4,65,number-card-9997,33330.0


In [78]:
# Concatenar do dataframe card_df para o dataframe df:
df = pd.concat([df.drop(['card'], axis=1), card_df], axis=1)
df.head()


,id,name,features,news,id,number,agency,balance,limit,card_id,card_number,card_limit
0,1,Devweekerson,"[{'id': 1, 'icon': 'https://digitalinnovationo...","[{'id': 1, 'icon': 'https://digitalinnovationo...",1,01.097954-4,2030,624.12,1000.0,1,xxxx xxxx xxxx 1111,2000.0
1,27,Predu,[],"[{'id': 170, 'icon': None, 'description': 'Pre...",34,99999-07,0007,4000.00,10000.0,27,**** **** **** 7777,10000.0
2,28,Pamonha,[],"[{'id': 171, 'icon': None, 'description': 'Oi ...",35,100000-08,0007,4000.00,10000.0,28,**** **** **** 8888,10000.0
3,29,Faustão,[],"[{'id': 172, 'icon': None, 'description': 'Olá...",36,100000-09,0007,4000.00,10000.0,29,**** **** **** 9999,10000.0
4,65,devteste-9997,"[{'id': 33, 'icon': 'string', 'description': '...","[{'id': 1022, 'icon': 'https://digitalinnovati...",72,number-9997,ag-9997,1230.00,1230.0,65,number-card-9997,33330.0


In [ ]:
# Renomear a coluna ID da conta para faciliar a identifição

df.rename()

In [79]:
# Realizar remoção da coluna Features e News:
df = df.drop(['features', 'news'], axis=1 )
df

,id,name,id,number,agency,balance,limit,card_id,card_number,card_limit
0,1,Devweekerson,1,01.097954-4,2030,624.12,1000.0,1,xxxx xxxx xxxx 1111,2000.0
1,27,Predu,34,99999-07,0007,4000.00,10000.0,27,**** **** **** 7777,10000.0
2,28,Pamonha,35,100000-08,0007,4000.00,10000.0,28,**** **** **** 8888,10000.0
3,29,Faustão,36,100000-09,0007,4000.00,10000.0,29,**** **** **** 9999,10000.0
4,65,devteste-9997,72,number-9997,ag-9997,1230.00,1230.0,65,number-card-9997,33330.0
5,99,KAIO,1086,0101-00,000-1,2000.00,0.0,997,9999 9999 9989 9999,1000.0



## Load
Carregar os dados em um novo arquivo Excel ou csv no diretório corrente para integração com Powerbi ou outra ferramenta de visualização para gerar relatórios/dashboards.


In [86]:
# Determinando o nome do Arquivo:
filename = 'customers_add_data.xls'
# salvando excel:
df.to_excel(filename)


<ipython-input-86-48316a91320c>:4: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel(filename)
